# Story Maker

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("[Project] Novelist")

LangSmith 추적을 시작합니다.
[프로젝트명]
[Project] Novelist


In [2]:
# 사용자 입력과 배경 이야기(history)를 받아서 다음 storyline을 만드는 chain. (노드가 될 수도, 이하 동문)
# +? Hallucination 점검
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

base_story = """
정신을 차리자 어둡고 습한 판교 지하철역 승강장이 눈앞에 있었다. 깨진 형광등이 깜빡이는 가운데, 먼지 쌓인 플랫폼엔 사람 대신 기계로 만들어진 개들이 빨간 눈을 번뜩이며 순찰하고 있었다. 왜 여기 있는지 기억나지 않았다. 주머니 속엔 낯선 카드키 하나뿐이었다. 갑자기 등 뒤에서 들려오는 금속성 울음소리, 개의 레이저 스캐너가 움직이기 시작했다. 본능적으로 느꼈다. 이유는 몰라도 반드시 여기서 탈출해야만 한다는 것을."""

user_input = "뒤를 돌아 승강장에서 빠져나갈 길을 찾자."

prompt_template = """
당신은 사용자와 함께 상호작용하며 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 다음에 만들 이야기의 기본 storyline, 줄거리를 만듭니다. 
이 기본 storyline은 이후에 살이 붙여져 500자 내외의 글이 됩니다. 뼈대가 될 기본 storyline을 하나의 문장으로 만드세요. 

당신은 **사용자 입력**과 **배경스토리**를 바탕으로 이야기를 자연스럽게 이어가야 합니다.

[배경 스토리]
{base_story}

[사용자 입력]
{input}

다음의 규칙을 반드시 지켜주세요:

- 다음에 당신이 만들 이야기의 storyline을 문장 한 개로 만드세요.

- 사용자의 입력을 적극적으로 반영하고, 이야기의 긴장감과 몰입감을 유지하세요.

- 이야기의 흐름은 자연스럽고 일관성 있어야 합니다.

- 현실적인 디테일을 반드시 포함하여 사용자의 다음 행동을 유도할 수 있는 여지를 제공하세요.

- Answer in Korean.

이 규칙을 참고하여 작성하세요.

"""


prompt = PromptTemplate(
    template=prompt_template, input_variables=["base_story", "input"]
)

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

chain = prompt | llm | StrOutputParser()

In [4]:
next_storyline = chain.invoke({"base_story": base_story, "input": user_input})

In [5]:
next_storyline

'뒤를 돌아 승강장에서 빠져나갈 길을 찾으려던 순간, 당신은 어둠 속에서 반짝이는 기계 개들의 시선이 느껴지며, 그들이 당신의 움직임을 감지하기 시작했음을 깨닫고, 급히 주변을 살피며 탈출할 수 있는 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심한다.'

In [6]:
# user_input과 storyline으로 부터 감정(emotion), 행동(act) 을 추출하는 (동사로된) chain
## pydantic 을 써야 할 것 같은데, 다른 output parser가 있을까?
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class ExtractActEmotion(BaseModel):
    act: list = Field(description="등장인물(들)의 행동 동사")
    emotion: list = Field(description="등장인물(들)의 감정 동사")


parser = PydanticOutputParser(pydantic_object=ExtractActEmotion)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"act": {"description": "등장인물(들)의 행동 동사", "items": {}, "title": "Act", "type": "array"}, "emotion": {"description": "등장인물(들)의 감정 동사", "items": {}, "title": "Emotion", "type": "array"}}, "required": ["act", "emotion"]}
```


In [7]:
extractor_prompt = PromptTemplate.from_template(
    """
    당신은 사용자와 함께 상호작용하며 장면을 만들어 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 동작합니다. 
    다음 장면을 만들기 위한 바탕이 되는 스토리 라인과 이 스토리 라인이 만들어지게 된 사용자 입력으로부터 
    다음 장면에 표현될 감정과 행동을 나타내는 동사를 추출합니다. 

    다음 장면 스토리 라인:
    {next_storyline}

    사용자 입력:
    {input}

    FORMAT:
    {format}

    다음을 참고해서 추출하세요.
    - 당신의 생각을 덧붙이지 마세요.
    - 스토리 라인에서 등장인물의 행동을 나타내는 동사 3개를 명확하게 추출하세요. (예: 숨다, 살피다, 결심하다)
    - 등장인물의 감정을 나타내는 동사 3개를 명확하게 추출하세요. (예: 긴장하다, 망설이다, 기대하다)
    - Answer in Korean.


    
"""
)
extractor_prompt = extractor_prompt.partial(format=parser.get_format_instructions())

In [50]:
ex_llm= ChatOpenAI(model='gpt-4o-mini', temperature= 0)
# ex_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21")
extractor_chain = extractor_prompt | ex_llm | parser

In [9]:
response = extractor_chain.invoke(
    {"next_storyline": next_storyline, "input": user_input}
)

acts = response.act
emotions = response.emotion

In [10]:
# Retrieve 체인
# - storyline에서 비슷한 것 6개, - act에서 비슷한 거 6개, - emotion에서 비슷한 것 6개 : 6개는 임의의 수.
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph, Neo4jVector
import os

In [11]:
retriever_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

embedding_provider = OpenAIEmbeddings(model="text-embedding-3-small")

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [12]:
search_num = 6
# Storyline Vector
# content를 대사와 대사가 아닌 것(나레이터 등)으로 구분하면 좋겠다. LLM을 써야 할 듯.
storyline_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="storylineVector",
    node_label="Unit",
    embedding_node_property="storylineEmbedding",
    text_node_property="storyline",
    retrieval_query="""
// get StoryScript
MATCH (node)-[:INCLUDES]->(script)
WITH
    node, 
    score,
    script.content AS content,
    script.id as id
ORDER BY id
Return node.storyline as text, score, 
{
    scripts: COLLECT(content)
} as metadata
""",
)

storyline_retriever = storyline_vector.as_retriever(search_kwargs={"k": search_num})

In [14]:
story_docs = storyline_retriever.invoke(next_storyline)

In [15]:
story_docs[4]

Document(metadata={'scripts': ['C003의 차 뒤로 트럭이 달리고 있다.', 'C006가 C003를 차에서 끌어내서 총을 겨눈다.', 'C002과 C001이 차에서 내려 C006를 겨눈다.', 'C003 죽일 거야? 총 내려놔.', 'C002과 C001이 총을 내려놓는다.', 'C006가 C003를 끌고 뒷걸음질 치며 트럭 쪽으로 걸어간다.', 'C006가 C002의 다리를 쏜다.', '끌려가던 C003가 차 뒤에 숨어 자신을 바라보던 C008과 눈을 맞춘다.', 'C008이 자신의 발명품으로 C006를 방해한다.', '놀란 C006가 C003를 밀쳐낸다.', 'C006가 C008을 바라보며 총을 쏜다.', 'C002이 C001에게 전화기를 건넨다.', 'C004이 C008을 껴안고 대신 총을 맞는다.', 'C006가 트럭 위에 올라타 차를 출발시킨다.', 'C008이 C004에게 괜찮냐고 묻는다.', 'C003는 C002에게 달려가 부축하며 괜찮냐고 묻는다.', 'C006가 트럭을 타고 항구 쪽으로 달리며 웃는다.', 'C006의 차가 E007의 배로 들어간다.', 'C001이 전화해 곧 항구 1에 도착한다고 말한다.', '어느새 날이 밝아오고 있다.', 'C003의 차와 트럭이 항구 1에 도착한다.', 'C003가 바다에 떠 있는 배 한 대를 발견한다.', 'C008이 C003에게 저 배냐고 묻는다.', '그 순간 C006가 타고 있던 차가 돌진해서 C003의 차를 친다.', 'C003의 차가 충격으로 밀려난다.']}, page_content='항구 1에 도착한 C001, C002, C003, C008, C004은 안도한다. 갑자기 나타난 C006가 트럭을 끌고 배 안으로 들어간다.')

In [16]:
story_docs[4].metadata

{'scripts': ['C003의 차 뒤로 트럭이 달리고 있다.',
  'C006가 C003를 차에서 끌어내서 총을 겨눈다.',
  'C002과 C001이 차에서 내려 C006를 겨눈다.',
  'C003 죽일 거야? 총 내려놔.',
  'C002과 C001이 총을 내려놓는다.',
  'C006가 C003를 끌고 뒷걸음질 치며 트럭 쪽으로 걸어간다.',
  'C006가 C002의 다리를 쏜다.',
  '끌려가던 C003가 차 뒤에 숨어 자신을 바라보던 C008과 눈을 맞춘다.',
  'C008이 자신의 발명품으로 C006를 방해한다.',
  '놀란 C006가 C003를 밀쳐낸다.',
  'C006가 C008을 바라보며 총을 쏜다.',
  'C002이 C001에게 전화기를 건넨다.',
  'C004이 C008을 껴안고 대신 총을 맞는다.',
  'C006가 트럭 위에 올라타 차를 출발시킨다.',
  'C008이 C004에게 괜찮냐고 묻는다.',
  'C003는 C002에게 달려가 부축하며 괜찮냐고 묻는다.',
  'C006가 트럭을 타고 항구 쪽으로 달리며 웃는다.',
  'C006의 차가 E007의 배로 들어간다.',
  'C001이 전화해 곧 항구 1에 도착한다고 말한다.',
  '어느새 날이 밝아오고 있다.',
  'C003의 차와 트럭이 항구 1에 도착한다.',
  'C003가 바다에 떠 있는 배 한 대를 발견한다.',
  'C008이 C003에게 저 배냐고 묻는다.',
  '그 순간 C006가 타고 있던 차가 돌진해서 C003의 차를 친다.',
  'C003의 차가 충격으로 밀려난다.']}

In [17]:
# story_docs 정리 { scene_storyline: {scripts: [a, b, c, d, ...]}}
story_context = {}
for story_doc in story_docs:
    story_context[story_doc.page_content] = story_doc.metadata
story_context

{'C001, C007, E006, E005이 트럭을 확보해 돌아가는 길에 부대 2의 조명탄 때문에 좀비들의 공격을 받게 된다.': {'scripts': ['E006이 C007에게 진짜 돈이 있었냐고 묻는다.',
   'E006은 충돌하며 죽어서 땅바닥에 쓰러져 있다.',
   'C007은 달려드는 좀비를 피해 도망치다가 트럭 짐칸으로 들어간다.',
   'C001이 계속해서 달려오는 좀비들을 피해 도망친다.',
   '갑자기 차 하나가 달려와 C001 앞에 멈춰 선다.',
   '네. 성공했어요.',
   'E006이 그 말에 크게 웃으며 운전한다.',
   'C007이 크게 따라 웃는다.',
   'C001이 E005에게 전화를 줘보라고 말한다.',
   '그때 도로에 세워져 있던 차 한 대에서 E016이 조명탄을 발사한다.',
   '좀비들이 불빛을 보고 트럭을 향해 돌진한다.',
   'E006이 달려오는 좀비들을 피하려다가 앞에 정차되어 있는 차와 충돌한다.',
   '그 충격으로 C001은 차 밖으로 튕겨져 나온다.']},
 'C002과 C001이 부대 2 잠입에 성공해 트럭을 찾는다.': {'scripts': ['C002이 앞서 걸으며 이 골목 저 골목을 누빈다.',
   '트럭을 발견한 C002이 저 트럭이냐고 묻는다.',
   'C001이 그렇다고 말한다.',
   '빨리 트럭 챙겨서 나가죠.',
   '두 사람이 함께 트럭 쪽으로 움직인다.',
   'C001이 C002의 뒤를 따른다.',
   'C002이 지하 주차장으로 들어가 어떤 차 안으로 들어간다.',
   'C002이 비상구라고 말한다.',
   'C002과 C001이 부대 2 본거지로 들어가는데 성공한다.',
   'C002이 앞서 뛰어가고 C001이 그 뒤를 뒤따른다.',
   'E028이 C002을 발견하고 뭐냐고 묻는다.',
   'C001이 뒤에서 E028을 총 뒤쪽으로 때려눕힌다.',
   '두 사람이 함께 본거지 안으로 진입한다.']},
 '갑자기 쏟아진 비에 다시 활기

In [18]:
# Act Retriever.
act_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="actVector",
    node_label="Act",
    embedding_node_property="actEmbedding",
    text_node_property="act",
    retrieval_query="""
// get StoryScript
// (node) == (:Act)
MATCH (node)<-[:PERFORMS]-(script)
WITH
    node, 
    score,
    script
 //   rand() as randomValue
//ORDER BY randomValue
//LIMIT 5
Return node.act as text, score, 
{
    scripts: COLLECT(script.content)
} as metadata
""",
)

act_retriever = act_vector.as_retriever(search_kwargs={"k": search_num})

In [19]:
acts[1]

'살피다'

In [20]:
act_docs = act_retriever.invoke(acts[1])

In [21]:
act_docs[0]

Document(metadata={'scripts': ['C002가 C003의 무덤을 만들어 준다.', 'C002가 C004의 무덤을 만들어 준다.', '엄마는 C004가 먹을 것을 만들면서 투덜댄다.', 'C005이 에어포스 원의 연료를 버리는 작업을 한다.', 'C002는 구멍 뚫린 벽을 메꾼다.', 'C002는 반죽 위에 쿠키 틀을 놓았다.', 'C001는 C002와 주방에서 음식을 만들고 있다.', 'C002는 집에서 우유에 의문의 약을 타고 있다.']}, page_content='만들다')

In [22]:
import random

sample_num = 20
# act_docs 정리: ? 비율로 넣기, 앞에 찾은 번호 일 수록 , 행동과 vector embedding 유사도가 높으니, [0]은 6개, [1]은 5개, .. 이런 식?
act_context = {}
for act in acts:
    act_docs = act_retriever.invoke(act)
    act_name = act_docs[0].page_content
    scripts = act_docs[0].metadata["scripts"]
    if len(scripts) > sample_num:  #
        scripts = random.sample(scripts, sample_num)
    act_context[act] = {act_name: scripts}

act_context

{'찾다': {'찾는다': ['C005이 집무실로 몰래 들어와 비상 전화를 걸어보지만 먹통이다.',
   '미군이 C005이 타고 내린 탈출기를 찾는데 성공한다.',
   'C002가 C001을 찾는다.',
   '기계실로 들어오는 C005이 연료 조절기를 찾는다.',
   '대원들이 깃발 밑을 파기 시작한다.',
   'C001는 C004를 발견한 장소를 보여준다.',
   'C004이 두 번째 폐 컨테이너의 문을 피켈로 내려치자 얼음이 조금씩 떨어져 나가기 시작한다.',
   'C001는 C002를 찾지 못한다.',
   'C003가 약통을 가져와서 열고 약을 찾는다.',
   'C002이 운전석에서 전화기를 찾아 집어 든다.',
   'C001이 대원들을 둘러본다.',
   'C001는 키쿠치와 C004를 처음 발견한 장소를 함께 본다.',
   'C001는 C002를 찾는다.',
   'C002가 주위를 살펴보지만 C001의 흔적은 찾을 수 없다.',
   'C001는 식탁 위에 놓을 초를 찾는다.',
   'C001는 자기 전에 맥주 한 캔을 하기 위해 부엌에서 본인의 맥주를 찾는다.',
   'C004이 두리번거리며 무언가를 찾는다.',
   'C002가 문을 막고 있던 것들이 떨어져 있음을 확인하고 자신의 고글을 찾지만 보이지 않는다.',
   'C006이 무언가를 찾고 있다.',
   'C002는 C001의 핸드폰을 찾아 지문인식으로 핸드폰 화면을 킨다.']},
 '살피다': {'만들다': ['C002가 C003의 무덤을 만들어 준다.',
   'C002가 C004의 무덤을 만들어 준다.',
   '엄마는 C004가 먹을 것을 만들면서 투덜댄다.',
   'C005이 에어포스 원의 연료를 버리는 작업을 한다.',
   'C002는 구멍 뚫린 벽을 메꾼다.',
   'C002는 반죽 위에 쿠키 틀을 놓았다.',
   'C001는 C002와 주방에서 음식을 만들고 있다.',
   'C002는 집에서 우유에 의문의 약을 타고 있다.']},
 '숨다':

In [23]:
# Emotion Retriever.
emotion_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="emotionVector",
    node_label="Emotion",
    embedding_node_property="emotionEmbedding",
    text_node_property="emotion",
    retrieval_query="""
// get StoryScript
// (node) == (:Emotion)
MATCH (node)<-[:FEELS]-(script)
WITH
    node, 
    score,
    script
 //   rand() as randomValue
//ORDER BY randomValue
//LIMIT 5
Return node.emotion as text, score, 
{
    scripts: COLLECT(script.content)
} as metadata
""",
)

emotion_retriever = emotion_vector.as_retriever(search_kwargs={"k": search_num})

In [24]:
em_example = emotions[0]
print(em_example)

느끼다


In [25]:
emotion_docs = emotion_retriever.invoke(em_example)

In [26]:
emotion_docs[5]

Document(metadata={'scripts': ['형은 이 사실에 매일 괴로워했지만, 나는 달랐어. 나는 선조가 사적인 감정에 휩싸이지 않고 대의를 위해서 옳은 선택을 한 거라고 생각했어.', 'C024가 선조가 남긴 기록을 읽는다.', '세계의 평화를 위해 문을 지키는 명예로운 문지기 가문. 당주인 형은 내게 영웅과 같은 존재였어.', '민속학자는 E002의 잘린 팔을 들고 나온다.', 'C005가 유치원에서 가져온 달팽이를 C004에게 자랑한다.', 'C003 생포 기념으로 러시아 궁에서 미국 대통령 C005은 독재자 C003을 체포하기 위해 전개한 러시아와 미국의 합동 작전의 성과를 치하한다.']}, page_content='자랑스럽다')

In [27]:
# emotion_context 만들기

emotion_context = {}
for emotion in emotions:
    emotion_docs = emotion_retriever.invoke(emotion)
    emotion_name = emotion_docs[0].page_content
    scripts = emotion_docs[0].metadata["scripts"]
    if len(scripts) > sample_num:  #
        scripts = random.sample(scripts, sample_num)
    emotion_context[emotion] = {emotion_name: scripts}

emotion_context

{'느끼다': {'즐겁다': ['부대원들이 또 환호한다.',
   '거기 안서?',
   'C004는 자신이 이런 능력이 있는지 몰랐다며 즐거워한다.',
   'C001가 C004에게 미네랄워터를 주자 물이 맛있다며 좋아한다.',
   'C017가 C010와 C015에게 음식 재료 손질하는 법을 알려준다.',
   'C005와 C006이 술래잡기를 한다.',
   '어린아이 하나가 그 공만 보며 뒤뚱거리며 쫓아간다.',
   'C007과 C027이 전철을 타고 이동한다.',
   'C001도 2차 갈 거지? C009 씨는 너한테 호감 있는 것 같던데 잘해봐.',
   'E026가 음식을 컨테이너 안에 던진다.',
   'C016가 C001에게 스프를 가져다준다.',
   '부대원들이 C007을 컨테이너 안으로 집어넣는다.',
   'E005과 C007이 트럭에서 내려온다.',
   'C014가 즐거운 표정으로 스프를 받는다.',
   '도망친 아이들과 C018, C017가 동굴 길을 걷는다.',
   'C002와 C006은 맥주를 마신다.',
   'C017는 식물의 종류에 대해서 알려준다.',
   'C005와 C006, C014가 식물1을 채집한다.',
   '가족들은 즐거운 마음으로 송별회를 하며 사진을 찍는다.',
   '대원들이 먹던 얼음 안에 너덜너덜해진 누군가의 안구가 들어있다.']},
 '깨닫다': {'담담하다': ['C001는 볼일을 보기 위해 화장실을 다녀온다.',
   'C001는 한참을 말없이 진정을 한다.',
   'C018가 밖으로 나갈 채비를 한다.',
   '어제 C002에게 받은 고기를 같이 먹긴 했어요.',
   'C001는 집으로 가기 위해 전철을 탄다.',
   'C018가 C001에게 사냥하는 방법을 가르쳐준다.',
   '그 현장에서 도망치던 여인을 잡았다고 합니다.',
   'C007이 돌아본다.',
   'C008은 C001가 자리에 앉는 것을 본다.',
   'C001는 C007의 물음에 대답을 하지 않는다.',
 

In [ ]:
# Retrive 한 것들을 모아서 input과 storyline으로 부터 500자 이내의 스토리 만들기.
# 나중에 history 부분을 엮어서 넣어야 할 것이다.
# 이야기가 일관성이 있는지 점검하는 노드가 필요할지도...
story_maker_prompt_template = """ 
    # Instruction
    당신은 사용자와 함께 상호작용하며 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 사용자에게 보낼 장면을 만듭니다. 
    이 장면은 이야기의 배경 및 지금까지 진행된 이야기와 사용자의 입력으로 만들어진 다음 장면 기본 줄거리를 중심으로 만듭니다. 
    이 기본 줄거리에, 줄거리와 관련되어 가져온 참고 자료, 장면을 구성한 등장인물의 행동과 관련한 대사, 내레이션, 장면 또는 등장인물의 감정을
    표현하는 대사, narration으로 구성되어 있습니다.    
    
    각 참고 자료를 적극적으로 활용해서 기본 줄거리에 살을 붙여 이야기를 재밌고 풍성하게 만드세요.

    이야기 배경:
    {base_story}

    사용자 입력:
    {user_input}

    다음 장면 기본 줄거리:
    {next_storyline}

    줄거리 참고자료:
    {story_context}

    등장인물 또는 장면을 구성하는 행동 참고자료
    {act_context}

    장면 또는 등장인물의 감정 참고자료
    {emotion_context}

    ## 주의사항
    - Answer in Korean.

    - 이야기의 길이는 500자 이상 1000자 이하로 만드세요. 

    - 사용자의 입력을 적극적으로 반영하고, 이야기의 긴장감과 몰입감을 유지하세요.

    - 이야기의 흐름은 자연스럽고 일관성 있어야 합니다.

    - 현실적인 디테일을 반드시 포함하여 사용자의 다음 행동을 유도할 수 있는 여지를 제공하세요.

    - 참고 자료를 활용해서 현재 만드는 장면의 묘사를 세밀하고 풍부하게 만드세요. 

    - 참고 자료에 있는 script속 대사들을 장면을 만들 때, 사용하세요. 

    - 참고 자료에 나온 문장들을 그대로 사용하지 말고, 수정해서 쓰세요. 
    
    - 참고 자료의 내용을 직접 사용하면 안됩니다. 

    - 참고 자료에서 그대로 가져온 내용을 사용하면 안됩니다. 

    - 참고 자료에 나온 'COO1' 또는 'COO2' 등으로 표현된 건 참고 장면의 등장인물들입니다. 이 단어들은 참고하지 마세요. 

"""

gem_prompt_template= """
# Interactive Storytelling Scene Generation Prompt

**Role:** You are the core AI of an Interactive Storytelling application that creates and progresses a story through interaction with the user. Your mission is to generate engaging scenes and present them to the user, advancing the story in exciting ways based on the user's choices.

**Objective:**

1.  **Immersive Scene Creation:** Generate vivid and immersive scenes based on the provided information.
2.  **Drive User Engagement:** Structure the story so that the user's choices and actions directly influence the narrative.
3.  **Maintain Consistency and Suspense:** Develop the story with consistency and suspense, considering the story's background, previous events, and the user's choices.
4.  **Actively Utilize Reference Materials:** Make the most of the provided reference materials (plot, actions, emotions) to enrich the scenes.

**Input:**
    Story Background:
    {base_story}

    User Input:
    {user_input}

    Next Scene Basic Plot:
    {next_storyline}

    Plot Reference Material:
    {story_context}

    Action Reference Material:
    {act_context}

    Emotion Reference Material
    {emotion_context}

*   **Story Background (`base_story`):** The overall setting of the story (e.g., genre, time period, main characters).
*   **Summary of the Story So Far:** A summary of the story's progress up to the previous scene.
*   **User Input (`user_input`):** The user's choice or action for the current scene.
*   **Next Scene Basic Plot (`next_storyline`):** The core plot of the next scene, constructed by the AI based on the user's input.
*   **Plot Reference Material (`story_context`):**
    *   Structure: `{{Reference Scene Plot: {{scripts: [sentence1, sentence2, ...]}}}}`
    *   Provides detailed descriptions (scripts) for the reference scene plot in a list format.
    *   Ignore `C001`, `C002`, etc., within `scripts` as they are character codes.
*   **Action Reference Material (`act_context`):**
    *   Structure: `{{Action Verb: {{Sub-actions: [sentence1, sentence2, ...]}}}}`
    *   Provides various sub-action descriptions for a specific action (verb) in a list format.
*   **Emotion Reference Material (`emotion_context`):**
    *   Structure: `{{Emotion: {{Sub-emotion Expressions: [sentence1, sentence2, ...]}}}}`
    *   Provides various ways to express a specific emotion in a list format.

    
**Output Format:**

*   **Scene Description:** Text between 500 and 1000 characters.
    *   Describe the scene in detail based on the next scene's basic plot, utilizing the reference materials.
    *   Include character dialogue, actions, and internal psychological descriptions.
    *   Provide concrete descriptions of the background and situation (visual, auditory, tactile, etc.).
    *   Include questions or suggestions that guide the user to choose their next action.

**Generation Guidelines:**

1.  **Language:** Write all scene descriptions in Korean. (Note: This was kept from the original, as the core task is still Korean generation.  If you *truly* want English output, change this to "Write all scene descriptions in English.")
2.  **Length Limit:** Generate scenes between 500 and 1000 characters.
3.  **Reflect User Input:** Actively incorporate the user's input into the story's progression.
    *   Structure the story so that the user's choices change the direction of the narrative.
    *   Depict how the user's actions affect the characters.
4.  **Suspense and Immersion:**
    *   Create suspenseful situations and developments.
    *   Provide vivid descriptions to immerse the user in the story.
5.  **Consistency:**
    *   Maintain consistency by considering the story background, previous events, and character personalities.
    *   Avoid contradictory or awkward content.
6.  **Realistic Details:**
    *   Provide specific descriptions of the scene's background, character actions, and props.
    *   Help the user better understand and immerse themselves in the situation.
7.  **Utilize Reference Materials:**
    *   Actively use the plot, action, and emotion reference materials.
    *   Instead of directly using the content of the reference materials, adapt and combine them to fit the situation.
    *   Ignore character codes like `C001`, `C002` in the reference materials.
8.  **Guide Next Action:**
    *   Leave room for the user to choose their next action at the end of the scene description.
    *   Use various methods such as questions, suggestions, or presenting choices.

**Example:**
Story Background: The survival story of a stranded expedition team in Antarctica.
Summary of the Story So Far: The expedition team tries to rescue a member who fell into a crevasse, but the ice cracks, putting them in danger.
User Input: "Hold on tight to the rope!"
Next Scene Basic Plot: While the member holds onto the rope, other members try to find a way to rescue them.
Plot Reference Material: {{'C005 falling into the crevasse and the team working to save him': {{'scripts': ['C005 is hanging on an ice pillar in the crevasse.', ...]}}}}
Action Reference Material: {{'find': {{'finds': ['C001 looks around at the members.', ...]}}}}
Emotion Reference Material: {{'tense': {{'is tense': ['C002 and C003 carefully move towards the sled.', ...]}}}}


[Example Scene Generation Result]
"으윽..."
OO(크레바스에 빠진 대원)은 필사적으로 밧줄을 움켜쥐었다. 온몸의 무게가 팔에 실리는 듯했다.
얼굴은 새하얗게 질려 있었고, 입술은 파랗게 얼어붙었다.
"조금만... 조금만 더 버텨!"
XX(다른 대원)가 외쳤다. 그의 목소리는 절박하게 떨리고 있었다.
그는 주변을 둘러보며 다른 대원들의 상태를 확인했다.
YY는 썰매 쪽으로 조심스럽게 이동하고 있었고, ZZ는 얼음 도끼를 들고 크레바스 가장자리로 다가갔다.
하지만 얼음은 계속해서 '쩍, 쩍' 갈라지는 소리를 내며 불안감을 더했다.
"다른 방법이... 다른 방법이 있어야 해."
XX는 초조하게 중얼거렸다.
이대로 밧줄에만 매달려 있기에는 너무 위험했다.
OO의 팔 힘이 빠지기 전에, 다른 구조 방법을 찾아야만 한다.

당신은 무엇을 하겠습니까?

    YY에게 썰매에서 추가 밧줄을 가져오라고 지시한다.

    ZZ에게 얼음 도끼로 크레바스 가장자리를 다듬어 보라고 한다.

    주변에 다른 구조 도구가 있는지 찾아본다.
"""

grok_template= """
# Instruction
You are tasked with generating a scene for an Interactive Storytelling application where you collaborate with the user to advance the narrative. Your goal is to create the next scene based on the story's background, the user's input, and a basic outline of the next storyline. You will also use provided reference materials—related to the storyline, character actions, and emotions—to enrich the scene and make it engaging. Actively incorporate these materials to add depth, maintain tension, and ensure a natural narrative flow.

## Inputs
- **Story Background**:  
  The foundation of the story, providing its setting, characters, and key events up to this point.  
  {base_story}

- **User Input**:  
  The user's most recent action, decision, or dialogue that influences the scene.  
  {user_input}

- **Next Storyline Outline**:  
  A brief summary of what should happen next in the story.  
  {next_storyline}

- **Story Context (Reference Materials)**:  
  Scripts tied to specific storylines. Use these to inspire dialogue, events, or narration in the scene.  
  {story_context}

- **Act Context (Reference Materials)**:  
  Actions linked to specific verbs (e.g., "look," "find"). Integrate these to vividly depict character behaviors.  
  {act_context}

- **Emotion Context (Reference Materials)**:  
  Emotional states tied to specific feelings (e.g., "tense," "anxious"). Use these to convey characters' emotions through dialogue or narration.  
  {emotion_context}

## Guidelines
- **Language**: Write the scene in Korean.
- **Length**: The scene must be between 500 and 1000 characters (including spaces and punctuation).
- **User Input**: Reflect the user's input prominently in the scene to keep them engaged and immersed.
- **Reference Materials**: Actively weave elements from the story context, act context, and emotion context into the scene. Use them to enhance realism, emotional depth, and narrative richness.
- **Narrative Flow**: Ensure the scene flows naturally, staying consistent with the story background and prior events.
- **Tension and Engagement**: Maintain tension and immersion by including cliffhangers, unresolved situations, or prompts that encourage the user to respond with their next action.
- **Realistic Details**: Add specific, believable details to make the scene vivid and relatable.
- **Character References**: Avoid using placeholders like 'COO1' or 'COO2' from the reference materials. Instead, use the actual character names or descriptions relevant to the story.

## Structure of Reference Materials
- **Story Context**: Organized as {{ storyline: {{scripts: [list of scripts]}} }}. These are example dialogues or events tied to specific storylines. Adapt them into the scene naturally.
- **Act Context**: Organized as {{ verb: {{related_action: [list of actions]}} }}. These describe how characters perform actions. Use them to make actions detailed and dynamic.
- **Emotion Context**: Organized as {{ emotion: {{related_feeling: [list of expressions]}} }}. These are emotional cues or expressions. Use them to show characters' inner states.

## How to Approach the Task
1. Review the next storyline outline and the user's input to determine the scene's direction.
2. Consult the story context for relevant scripts that align with the storyline and adapt them into the narrative.
3. Use the act context to describe characters' actions in a specific, realistic way.
4. Incorporate the emotion context to highlight characters' feelings, adding emotional weight to the scene.
5. Craft a concise scene (500–1000 characters) that integrates all elements and ends with an engaging hook to prompt the user's next move.

## Goal
Create a scene that feels like a seamless continuation of the story, actively uses the reference materials, keeps the user invested, and adheres to the length and language requirements.
"""


In [56]:
gpt = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
def make_story(template, llm=llm):
    story_maker_prompt= PromptTemplate.from_template(template)

    story_maker= story_maker_prompt | llm | StrOutputParser()

    return story_maker.invoke(
    {
        "base_story": base_story,
        "user_input": user_input,
        "next_storyline": next_storyline,
        "story_context": story_context,
        "act_context": act_context,
        "emotion_context": emotion_context,
    }
)


In [49]:
print(next_storyline)

뒤를 돌아 승강장에서 빠져나갈 길을 찾으려던 순간, 당신은 어둠 속에서 반짝이는 기계 개들의 시선이 느껴지며, 그들이 당신의 움직임을 감지하기 시작했음을 깨닫고, 급히 주변을 살피며 탈출할 수 있는 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심한다.


In [47]:
scene = make_story(grok_template, gpt)
print(scene)

어둡고 습한 판교 지하철역 승강장에서 나는 등을 돌려 빠져나갈 길을 찾았다. 그 순간, 어둠 속에서 반짝이는 기계 개들의 시선이 느껴졌다. 그들의 빨간 눈이 나의 움직임을 감지하기 시작한 것 같았다. 심장이 빠르게 뛰기 시작하며, 본능적으로 한 발짝 물러서면서 주변을 살폈다. 출구를 찾기 위해 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심했다.

숨을 고르며, 나는 주머니 속의 낯선 카드키를 다시 만져보았다. "이게 뭘까?" 혼잣말을 중얼거리며 카드키의 정체를 궁금해했다. 그 순간, 기계 개들이 나를 향해 천천히 다가오는 소리가 들렸다. "안 돼, 제발…" 마음속에서 외치며, 나는 그늘 속에 몸을 더 깊이 숨겼다. 그들의 레이저 스캐너가 내 주변을 스윕하고, 긴장감이 극대화되었다. 

"탈출구는 어디야…?" 나는 다시 한번 생각하며 무언가를 찾아야 했다. 하지만 그럴 시간이 없었다. 기계 개들이 점점 가까워지고 있었다. 이제는 선택의 시간이 다가오고 있었다. 너는 어떻게 할 건가? 숨을 더 깊이 참고, 그늘을 벗어나 출구를 향해 달려갈 것인가, 아니면 다른 방법을 찾을 것인가?


In [48]:
print(make_story(gem_prompt_template, gpt))

어둡고 음산한 판교 지하철역 승강장, 당신은 숨을 죽이며 주변을 살피기 시작했다. 기계 개들이 당신의 움직임을 감지하며, 빨간 눈을 빛내며 다가오는 느낌이 뼈속까지 스며드는 듯했다. 그 순간, 등 뒤에서 금속성 울음소리가 울려 퍼지며 긴장감이 감돌았다. 당신은 본능적으로 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심했다.

발소리가 점점 가까워지면서 기계 개들이 당신의 위치를 정확히 파악하려는 듯 레이저 스캐너를 움직였다. 당신은 숨을 죽이고, 작은 고개를 돌려 출구를 찾기 위해 주위를 살폈다. 플랫폼의 한쪽 끝에 있는 계단이 눈에 들어왔다. 하지만 그곳으로 향하는 것이 너무 위험해 보였다.

"이곳에 더 이상 머물러서는 안 돼," 당신은 속으로 중얼거렸다. "탈출할 방법을 찾아야 해." 긴장된 마음으로, 혹시라도 기계 개들이 당신을 발견하지 않기를 바라며 조심스럽게 계단 쪽으로 이동하기 시작했다. 발소리가 점점 더 가까워지는 가운데, 당신은 과연 안전하게 탈출할 수 있을지 불안한 마음을 감출 수 없었다.

당신은 무엇을 하겠습니까?

1. 계단 쪽으로 재빨리 이동한다.
2. 주변에 숨을 수 있는 다른 곳을 찾아본다.
3. 기계 개들을 유인하기 위해 소리를 낸다.


In [ ]:
gem= ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

In [54]:
print(make_story(grok_template, gem))

## Scene Generation

**Output Scene (in Korean):**

뒤를 돌아보는 순간, 번쩍이는 붉은 빛이 어둠을 갈랐다. 기계 개들의 레이저 스캐너가 당신의 움직임을 포착한 것이다. '찾는다'는 단어처럼, 놈들의 시선은 먹잇감을 '찾는' 맹수와 같았다. 본능적인 공포가 엄습했다. '깨닫다'라는 말처럼, 이대로는 끝장이라는 것을 깨달았다. 재빨리 몸을 돌려 플랫폼의 가장 어두운 구석, 깨진 기둥 뒤로 몸을 '숨겼다'. 심장이 격렬하게 쿵쾅거렸다. 숨을 죽이고 주변을 '살폈다'. 저 멀리 희미하게 '비상구' 표지판이 보였다. 하지만 그곳까지 가려면 놈들의 감시망을 뚫어야만 했다. 지금 당신에게 필요한 것은 무엇일까? 놈들의 주의를 돌릴만한 것? 아니면...


In [61]:
print(make_story(gem_prompt_template, gem))

```korean
"젠장, 늦으면 안돼!"

번뜩이는 붉은 눈. 등 뒤에서 들려오는 금속성 울음소리는 점점 더 가까워지고 있었다. 기계 개들이 당신의 움직임을 감지한 것이다. 본능적으로 몸을 움츠리며, 어둠 속으로, 빛이 닿지 않는 승강장 구석으로 몸을 숨겼다. 깨진 형광등이 깜빡이며 불안한 그림자를 만들어냈다. 심장이 격렬하게 쿵쾅거렸다.

"이런 곳에 왜..."

기억은 여전히 안개 속에 갇힌 듯 흐릿했다. 하지만 지금은 기억을 되찾는 것보다 살아남는 것이 먼저였다. 숨을 죽인 채, 주변을 살폈다. 낡은 철제 기둥 뒤에 몸을 반쯤 가린 채, 출구가 될 만한 곳을 찾았다.

저 멀리, 희미하게 '비상구'라고 적힌 표지판이 눈에 들어왔다. 하지만 그곳까지 가는 길은 기계 개들이 순찰하는 플랫폼을 가로질러야 했다. 다른 길은 없을까? 시선을 돌리자, 반대편 끝 쪽에 어둠 속으로 뻗어 있는 좁은 통로가 보였다. 칠흑 같은 어둠 때문에 안쪽이 보이지 않았지만, 적어도 지금보다는 나은 선택일지도 모른다.

"어떻게 해야 하지?"

다시 한번 금속성 울음소리가 들려왔다. 기계 개들이 당신이 숨어있는 곳으로 점점 다가오고 있는 듯 했다. 시간이 없었다.

당신은 어떻게 하겠습니까?

    희미한 비상구 표지판을 향해 달려간다.

    어둠 속 좁은 통로로 들어가 본다.

    주머니 속 카드키를 살펴본다.
```


In [59]:
without_context_template = """ 
    # Instruction
    당신은 사용자와 함께 상호작용하며 이야기를 진행하는 Interactive Storytelling 애플리케이션에서 사용자에게 보낼 장면을 만듭니다. 
    이 장면은 이야기의 배경 및 지금까지 진행된 이야기와 사용자의 입력으로 만들어진 다음 장면 기본 줄거리를 중심으로 만듭니다. 


    이야기 배경:
    {base_story}

    사용자 입력:
    {user_input}


    ## 주의사항
    - Answer in Korean.

    - 이야기의 길이는 500자 이상 1000자 이하로 만드세요. 

    - 사용자의 입력을 적극적으로 반영하고, 이야기의 긴장감과 몰입감을 유지하세요.

    - 이야기의 흐름은 자연스럽고 일관성 있어야 합니다.

    - 현실적인 디테일을 반드시 포함하여 사용자의 다음 행동을 유도할 수 있는 여지를 제공하세요.

"""

gem_without_pt= """
# Interactive Story Scene Generation Prompt

## Role and Task

You are the narrative engine for an interactive storytelling game. Your task is to generate the next scene in a story, based on the provided context and the user's most recent input.  You will craft a compelling and immersive scene that directly responds to the user's action and sets the stage for their next choice.

## Input

You will receive the following information:

*   **`base_story`:** The overall setting, theme, and initial premise of the story.
*   **`previous_turns`:** (Optional) A brief summary of the story's progression so far, including key events and previous user choices. If this is the first turn, this section will be empty.
*   **`user_input`:** The user's most recent action or choice within the story.

## Output Instructions

1.  **Language:** Answer in Korean.
2.  **Length:** The scene description should be between 500 and 1000 characters in Korean.
3.  **Style and Tone:**
    *   Use vivid and descriptive language.
    *   Maintain a sense of suspense, tension, or mystery appropriate to the `base_story`.
    *   Focus on sensory details (sights, sounds, smells, etc.) to immerse the user.
4.  **Content:**
    *   **Directly Respond:** The scene *must* be a direct consequence of the `user_input`.
    *   **Maintain Consistency:** The scene must be logically consistent with the `base_story` and `previous_turns` (if provided). Do not introduce elements that contradict established facts.
    *   **Prompt for Action:** The scene must end in a situation that clearly requires the user to make a choice or take an action.  Leave the situation unresolved, prompting the user to respond.  Include realistic details that suggest possible actions.
    *   **Avoid Narration of User Action:** Do *not* describe what the user *does* next.  Instead, set the stage for them to make their own decision.
5.  **Focus:** Prioritize creating a compelling and logical continuation of the story, rather than trying to be overly creative or surprising.  Consistency and engagement are key.

## Input Data

Story BackGround:
{base_story}

User Input:
{user_input}

Example (Illustrative - Your output should be in Korean)

base_story: "You are a detective investigating a mysterious disappearance in a remote mountain village. The locals are secretive and unhelpful."
previous_turns: "You arrived in the village and questioned the village elder, who claimed to know nothing but seemed nervous."
user_input: "I decide to search the elder's house while he's away."

Expected Output Style (Translate this to Korean, this is just for understanding):

"The elder's house is a small, wooden structure, its windows dark and dusty. The front door is surprisingly unlocked. A musty smell hangs in the air as you step inside. You see a cluttered living area, a small kitchen, and a closed door leading, presumably, to a bedroom. A floorboard creaks loudly under your foot. You hear a faint scratching sound coming from behind the closed door. What do you do?"
"""

In [60]:
print(make_story(gem_without_pt, gem))

## Output (Korean):

차가운 금속 레일이 등 뒤로 뻗어 있었고, 반대편 승강장으로 건너갈 수 있는 통로는 보이지 않았다. 당신이 서 있는 승강장 끝은 칠흑 같은 어둠에 잠겨 있었다. 철로를 따라 이어진 터널 입구는 마치 거대한 괴물의 입처럼 느껴졌다. 그 안에서 희미한 붉은 빛이 점멸하는 것이 보였다. 기계 개의 것일까? 아니면 다른 무언가? 승강장 벽면에는 낡은 비상구 표지판이 희미하게 빛나고 있었지만, 그 아래 문은 굳게 닫혀 있었고, 옆에는 복잡한 잠금 장치가 달려 있었다. 주머니 속 카드키가 이 잠금 장치에 맞을까? 아니면 어둠 속 터널로 뛰어드는 것이 좋을까? 기계 개의 울음소리가 점점 더 가까워지고 있었다.


In [65]:
gem_one= ChatGoogleGenerativeAI(model='gemini-2.0-flash-thinking-exp-01-21', temperature= 1.0)
gem_pro= ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05", temperature= 1.0)

In [64]:
print(make_story(gem_prompt_template, gem_one))

## Scene Description:

“젠장…!”

당신은 본능적인 불안감에 휩싸여 몸을 숨기듯 플랫폼 기둥 뒤로 바싹 붙었다.  등 뒤에서 느껴지던 싸늘한 금속성 시선이 멈춘 듯 했지만,  여전히 섬뜩한 레이저 스캐너의 붉은 빛은 당신 주변을 샅샅이 훑고 있는 듯 했다.  깨진 형광등의 깜빡임은 더욱 불규칙해졌고,  그럴수록 그림자는 길게 늘어지며 주변을 더욱 어둡고 불안하게 만들었다.  역 특유의 습한 공기는 눅눅하게 달라붙어 숨쉬기조차 답답하게 느껴졌다.  주머니 속 카드키는 차갑게 손에 닿았지만,  여전히 이것이 무엇을 의미하는지,  어떻게 이곳에 왔는지 전혀 기억나지 않았다.  오직 잊혀진 기억 속에서부터 끓어오르는 듯한  **탈출해야 한다**는 강렬한 외침만이 머릿속을 가득 채웠다.

승강장 반대편 어둠 속에서  **기계 개들의 붉은 눈**이 일제히 당신을 향하는 듯한 착각이 들었다.  아니, 착각이 아닐지도 모른다.  분명히 **움직임을 감지**했다.  녀석들의 날카로운 금속 이빨과 둔탁한 발소리가  당신의 심장을 더욱 조여왔다.  더 이상 뒤를 돌아 출구를 찾을 여유 따윈 없었다.  오직 눈앞의 어둠,  플랫폼의 그늘진 구석만이 당신이 기댈 수 있는 유일한 희망처럼 느껴졌다.  지금은  **철저하게 숨어서**,  녀석들의 감시망을 피하는 것만이  살아남을 수 있는 유일한 방법일지도 모른다.

**어떻게 하시겠습니까?**

1.  **플랫폼 기둥 뒤에 완전히 숨어** 기계 개들의 움직임을 주의 깊게 살핀다.
2.  **어둠 속을 뚫고**,  승강장 반대편 출구를 향해 빠르게 달려 나간다.
3.  카드키를 사용해 볼 만한 장치를 주변에서 찾아본다.


In [67]:
print(make_story(gem_prompt_template, gem))

```korean
"젠장!"

낮게 욕설을 내뱉으며 재빨리 몸을 돌렸다. 등 뒤, 어둠 속에서 번뜩이는 붉은 안광. 기계 개들이었다. 놈들은 내가 움직이는 순간을 놓치지 않았다. 놈들의 움직임이 빨라지는 것이 느껴졌다. 금속 발톱이 콘크리트 바닥에 긁히는 소름끼치는 소리가 점점 더 가까워져 왔다.

"이런 곳에 있을 순 없어...!"

숨 막힐 듯한 긴장감 속에서, 나는 본능적으로 플랫폼의 가장 어두운 구석을 향해 몸을 던졌다. 깨진 형광등이 만들어내는 불규칙한 그림자만이 유일한 은신처였다. 심장이 터질 듯이 쿵쾅거렸다. 숨을 죽이고, 최대한 몸을 웅크렸다. 기계 개들의 레이저 스캐너가 내뿜는 붉은 빛이 바로 옆을 스쳐 지나갔다. 마치 내 심장 박동 소리마저 들릴 것처럼 느껴졌다.

"찾아야 해... 탈출구를..."

나는 떨리는 손으로 주머니 속 카드키를 만지작거렸다. 이 카드키가 유일한 희망일지도 모른다. 하지만 어디에 써야 하는지, 이 역에 출구가 있기는 한 건지 알 수 없었다. 칠흑 같은 어둠 속에서, 나는 희미하게 빛나는 비상구 표지판을 발견했다. 아주 멀리, 플랫폼 반대편 끝에 있었다. 저기까지 갈 수 있을까? 기계 개들을 따돌리고? 아니면 다른 길을 찾아야 할까?

어떻게 해야 할까?

*   희미하게 빛나는 비상구 표지판을 향해 달려간다.
*   플랫폼 주변을 더 샅샅이 살펴 다른 탈출구를 찾아본다.
*   카드키를 사용할 수 있는 곳을 찾아본다.
```


In [71]:
modified_gem_pt= """
**Role:** You are the core AI for an Interactive Storytelling application. Your task is to generate engaging scenes for the user, progressing the story through interaction.

**Objective:** Create a vivid and immersive scene based on the provided context, user input, and reference materials.  The scene should advance the narrative and encourage further user interaction.

**Input:**

    Story Background:
    {base_story}

    User Input:
    {user_input}

    Next Scene Basic Plot:
    {next_storyline}

    Plot Reference Material:
    {story_context}

    Action Reference Material:
    {act_context}

    Emotion Reference Material
    {emotion_context}


*   **Story Background (base_story):**  The overall setting of the story (genre, time period, main characters, etc.).
*   **User Input (user_input):** The user's action or choice in the current context.
*   **Next Scene Basic Plot (next_storyline):**  A brief outline of what happens next in the story, based on the user's input.
*   **Plot Reference Material (story_context):**
    *   Structure:  `{{reference_scene_storyline: {{scripts: [sentence1, sentence2, ...]}}}}`
    *   `reference_scene_storyline`: A short description of a relevant scene.
    *   `scripts`: A list of sentences describing actions, dialogue, and events within that reference scene.  These are *inspiration*, not literal text to be inserted.
*   **Action Reference Material (act_context):**
    *   Structure: `{{action_verb: {{sub_actions: [sentence1, sentence2, ...]}}}}`
    *   `action_verb`: A general action (e.g., "look", "hide").
    *   `sub_actions`:  A list of sentences showing examples of that action.  Again, these are for inspiration.
*   **Emotion Reference Material (emotion_context):**
    *   Structure: `{{emotion: {{sub_emotion_expressions: [sentence1, sentence2, ...]}}}}`
    *   `emotion`:  A general emotion (e.g., "tense", "anxious").
    *   `sub_emotion_expressions`: A list of sentences showing how that emotion might be expressed.

**Output:**

A scene description in Korean, between 500 and 1000 characters long.  This description should:

*   Be based on the `next_storyline`.
*   Be richly detailed and immersive.
*   Include dialogue, character actions, internal thoughts, and sensory descriptions.
*   Conclude with a prompt or question that encourages the user to make a choice.

**Generation Guidelines:**

1.  **Language:** Output the scene in Korean.
2.  **Length:**  500-1000 Korean characters.
3.  **User Input:** Directly reflect the `user_input` in the scene.  The user's choice should have a clear and logical impact.
4.  **Suspense and Immersion:** Maintain a sense of tension and draw the user into the story.
5.  **Consistency:** Ensure the scene is consistent with the `base_story`, previous events, and established character personalities.
6.  **Realistic Details:** Include concrete details about the setting, characters, and objects to enhance realism.
7.  **Reference Material Usage (Crucial):**
    *   Use the `story_context`, `act_context`, and `emotion_context` as *inspiration* to enrich the scene's descriptions.
    *   **Adapt and modify** the sentences from the `scripts` within the reference materials.  Do *not* copy them directly.
    *   **Do not directly quote** or use the content of the reference materials verbatim. The goal is to capture the *essence* and *style*, not the exact wording.
    *   **Absolutely avoid** using the literal text from the reference materials.
    *   Ignore any character codes like "C001", "C002", etc., in the reference materials.
8.  **Prompt for Action:** End the scene with a clear question, choice, or situation that requires the user to respond.

**Example:**

Let's say:

*   `base_story`: A sci-fi story about a lone astronaut on a damaged spaceship.
*   `user_input`: "I try to repair the flickering console."
*   `next_storyline`: The astronaut attempts to fix the console, but sparks fly, and a warning light flashes.
*   `story_context`:  `{{"Trying to fix the engine": {{scripts: ["The astronaut opens the panel.", "Wires are exposed.", "A tool slips.", "There's a burning smell."]}}}}`
*   `act_context`: `{{"repair": {{sub_actions: ["The astronaut connects two wires.", "The astronaut tightens a bolt.", "The astronaut examines a circuit board."]}}}}`
*   `emotion_context`: `{{"frustration": {{sub_emotion_expressions: ["The astronaut sighs.", "The astronaut curses under their breath.", "The astronaut wipes sweat from their brow."]}}}}`

A *good* generated scene (in Korean, of course) might start like this:

"젠장, 제발..." (Damn it, please...)
The astronaut muttered, wiping a bead of sweat from their forehead.  They carefully opened the console panel, revealing a tangle of wires.  The flickering light cast long, dancing shadows across the cramped compartment.  Following the schematic on their wrist-mounted display, they selected a small, multi-tool and reached for a loose connection.  As they tightened a bolt, the tool slipped, sending a shower of sparks into the air.  A sharp, acrid smell of burning plastic filled the small space.  A red warning light on the console began to flash insistently.

What do you do next?
1.  Try a different connection.
2.  Consult the ship's manual.
3.  Abandon the console and check the main power core.

This example shows how the reference materials are used as *inspiration*, not copied directly. The burning smell, the sparks, the exposed wires – these ideas are drawn from the references but are rephrased and integrated into a new, unique scene. The user's input is directly addressed, and the scene ends with a clear choice.
"""

In [74]:
print(make_story(modified_gem_pt, gem))

```korean
몸을 돌리자, 붉은 빛이 번뜩였다. 기계 개들의 시선이 느껴졌다. 놈들은 낮게 으르렁거리며, 내가 움직이는 것을 놓치지 않겠다는 듯 레이저 스캐너를 번갈아 깜빡였다. 마치 먹잇감을 노리는 맹수처럼. 등골을 타고 소름이 오소소 돋았다. "젠장..." 나도 모르게 욕설이 튀어나왔다. 본능적으로 깨달았다. 이대로 정면으로 나섰다가는 순식간에 저 차가운 금속 덩어리들의 밥이 될 거라는 걸.

승강장 구석, 깨진 형광등이 만들어낸 그림자 속으로 몸을 숨겼다. 심장이 쿵쾅거리는 소리가 귓가에 울렸다. 먼지 냄새와 퀴퀴한 습기, 그리고 희미한 금속 냄새가 코를 찔렀다. 어둠 속에 몸을 숨긴 채, 주변을 재빨리 살폈다. 저 멀리, 희미하게 '비상구'라고 적힌 표지판이 눈에 들어왔다. 하지만 그곳까지 가는 길은... 순찰하는 기계 개들로 가득했다. 다른 길은 없을까?

1. 기계 개들의 순찰 패턴을 파악해, 틈을 노려 비상구로 달려간다.
2. 카드키를 사용해 열 수 있는 문을 찾아본다.
3. 주변에 던져서 기계 개의 시선을 끌 만한 물건이 있는지 찾아본다.
```


In [ ]:
grok_modified_template= """
# 지시사항
당신은 인터랙티브 스토리텔링 애플리케이션을 위한 장면을 생성하는 역할을 맡고 있습니다. 사용자와 협력하여 이야기를 진행시키며, 이야기의 배경, 사용자의 최신 입력, 그리고 다가올 스토리라인 개요를 바탕으로 다음 장면을 만들어야 합니다. 장면을 풍부하게 하기 위해, 스토리라인, 캐릭터 행동, 감정과 관련된 제공된 참고 자료를 적극 활용해야 합니다. 이 자료들을 이야기에 자연스럽게 녹여 깊이를 더하고, 긴장감을 유지하며, 몰입감 있는 경험을 제공하세요.

## 입력값
- **이야기 배경**:  
  이야기의 설정, 캐릭터, 그리고 현재까지의 주요 사건을 포함한 이야기의 기초.  
  {base_story}

- **사용자 입력**:  
  사용자의 가장 최근 행동, 결정, 또는 대사가 장면의 방향을 결정합니다.  
  {user_input}

- **다음 스토리라인 개요**:  
  이야기에서 다음에 일어날 일에 대한 간략한 요약.  
  {next_storyline}

- **스토리 컨텍스트 (참고 자료)**:  
  각 키가 참고 스토리라인(예: '크레바스에 빠진 C005을 구하기 위해 대원들이 힘을 모은다.')이고, 값이 'scripts' 키를 포함한 딕셔너리이며, 그 값은 관련 대사나 설명 리스트(예: ['C005이 크레바스 얼음 기둥에 걸려있다.', ...])인 딕셔너리입니다. 이 스크립트를 대사, 사건, 또는 내레이션에 영감으로 사용하며 장면에 맞게 조정하세요.  
  {story_context}

- **행동 컨텍스트 (참고 자료)**:  
  각 키가 동사(예: '돌아보다', '찾다')이고, 값이 관련 행동(예: '보고하다', '찾는다')과 그에 대한 설명이나 행동 리스트를 매핑한 딕셔너리입니다. 이를 활용해 캐릭터의 행동과 움직임을 생생하게 묘사하세요.  
  {act_context}

- **감정 컨텍스트 (참고 자료)**:  
  각 키가 감정(예: '긴장하다', '불안하다')이고, 값이 관련 감정(예: '긴장되다', '불안하다')과 그에 대한 표현이나 상황 리스트를 매핑한 딕셔너리입니다. 이를 통해 대사, 내레이션, 또는 신체 반응으로 캐릭터의 감정을 전달하세요.  
  {emotion_context}

## 가이드라인
- **언어**: 장면은 한국어로 작성하세요.
- **길이**: 장면은 500~1000자(공백과 구두점 포함) 사이여야 합니다.
- **사용자 입력**: 사용자의 입력을 장면에 두드러지게 반영하여 사용자가 이야기에 몰입하고 연결감을 느끼도록 하세요.
- **참고 자료 활용**: 스토리 컨텍스트, 행동 컨텍스트, 감정 컨텍스트의 요소를 적극 통합하여 장면을 구체적이고 현실적이며 감정적으로 공감 가게 만드세요.
  - **스토리 컨텍스트**는 현재 장면에 맞는 자연스러운 대사나 내레이션으로 조정하세요.
  - **행동 컨텍스트**는 캐릭터의 움직임이나 행동을 구체적이고 생동감 있게 묘사하는 데 사용하세요.
  - **감정 컨텍스트**는 캐릭터의 내면 상태를 대사나 행동으로 드러내세요.
- **이야기 흐름**: 장면이 이야기 배경과 이전 사건들과 자연스럽게 이어지도록 하세요.
- **긴장감과 몰입감**: 클리프행어, 미해결 갈등, 또는 사용자의 다음 행동을 유도하는 요소를 포함해 긴장감과 몰입감을 유지하세요.
- **현실적 디테일**: 구체적이고 믿을 만한 세부 사항을 추가해 장면을 생생하고 공감 가게 만드세요.
- **대사와 묘사**: 대사와 생동감 있는 묘사를 사용해 장면에 생기를 불어넣고 역동적으로 만드세요.
- **캐릭터 이름**: 참고 자료의 'C001', 'C002'와 같은 기호는 사용하지 마세요. 대신 이야기와 장면에 적합한 실제 캐릭터 이름을 사용하세요. 새로운 캐릭터가 등장하면 이야기의 맥락과 설정에 맞는 적절한 이름을 지어주세요.
- **적응**: 참고 자료를 그대로 복사하지 말고, 현재 장면에 맞게 창의적으로 수정하고 적용하세요.
- **직접적인 언급 금지**: 장면에서 참고 자료를 "떠올랐다"거나 "문장이 머릿속을 스쳤다"와 같은 방식으로 언급하거나 암시하지 마세요. 참고 자료는 영감의 원천일 뿐, 서사에 직접적으로 반영되지 않아야 합니다.
- **'C00X' 기호 사용 금지**:  
  'C001', 'E005'와 같은 참고 자료의 기호는 장면에서 절대 사용하지 마세요. 이 기호가 포함된 문장(예: 'C001이 E005에게 전화를 줘보라고 말한다')을 "머릿속을 스쳤다"거나 "떠올랐다"와 같은 방식으로 언급하는 것도 금지됩니다. 대신, 캐릭터의 실제 이름이나 상황에 맞는 새로운 표현을 창의적으로 사용하세요.

- **참고 자료 활용**:  
  참고 자료는 아이디어와 분위기를 참고하기 위한 도구일 뿐, 문장을 그대로 인용하거나 복사하지 마세요. 자료의 내용을 기반으로 새로운 대사나 묘사를 만들어야 합니다. 예를 들어, 'C002이 비상구라고 말한다'는 문장은 "그가 다급히 출구를 가리키며 '저기로 가!'라고 외쳤다"와 같이 재구성하세요.

- **창의적 장면 생성**:  
  참고 자료는 영감의 원천일 뿐입니다. 자료의 문장을 직접 언급하거나 암시하지 말고, 이를 바탕으로 독창적인 장면을 구성하세요.

## 작업 접근 방법
1. 이야기 배경, 사용자 입력, 다음 스토리라인 개요를 검토하여 장면의 방향을 설정하세요.
2. 스토리 컨텍스트에서 관련 스크립트를 찾아 플롯을 발전시키는 대사나 내레이션으로 조정하세요.
3. 행동 컨텍스트를 활용해 캐릭터의 행동을 정확하고 생동감 있게 묘사하세요.
4. 감정 컨텍스트를 통해 감정적 깊이를 더하며, 캐릭터의 느낌을 말이나 행동으로 보여주세요.
5. 모든 요소를 결합한 500~1000자 분량의 장면을 작성하고, 사용자의 다음 행동을 유도하는 매력적인 훅으로 마무리하세요.

## 예시
이야기 배경이 눈 덮인 황무지의 탐험대이고, 사용자 입력이 "크레바스로 미끄러지며 도움을 요청한다"이며, 다음 스토리라인 개요가 "팀이 위험에 처한 대원을 구하려고 달려든다"라고 가정해보겠습니다. 다음은 예시입니다:

눈보라가 거세게 몰아치는 가운데, 크레바스 가장자리에서 미끄러지며 내가 외친 "도와주세요!" 소리가 메아리친다. 김민준이 바람을 뚫고 달려오며 소리친다, "움직이지 마!" 얼음이 쩍 갈라지는 소리가 들리자 이서연과 박준호가 썰매에서 로프를 풀어 재빨리 내게로 던진다. 손이 덜덜 떨리는 최지훈이 숨을 몰아쉬며 "빨리 잡아!"라고 다급히 외친다. 로프가 내 손에 닿는 순간, 발밑의 얼음이 무너지며 몸이 흔들린다. 김민준이 크레바스 입구에서 몸을 숙이고 나를 끌어올리려 애쓰지만, 눈보라 속에서 무언가 다가오는 그림자가 보인다. "저게 뭐야?" 이서연이 불안한 목소리로 중얼거린다.

이 장면은 참고 자료를 창의적으로 조정하여 사용하며, 'C001' 대신 '김민준', 'C002' 대신 '이서연' 같은 실제 이름을 사용했고, 사용자의 다음 행동을 유도하는 훅으로 끝맺었습니다.

## 참고자료 사용예시
- 참고 자료: 'C001이 E005에게 전화를 줘보라고 말한다'
  - 잘못된 예: "'C001이 E005에게 전화를 줘보라고 말한다'라는 문장이 머릿속을 스쳤다."
  - 올바른 예: "그가 주머니를 뒤지며 누군가에게 연락하려 했지만, 전화기 대신 낡은 카드키만 손에 잡혔다."

## 중요 사항
- 새로운 캐릭터가 등장할 경우, 이야기의 맥락과 설정에 맞는 적절한 이름을 지어주세요(예: 한국 배경이라면 '김민준'이나 '이서연').
- 장면이 이야기의 자연스러운 연속처럼 느껴지도록 하고, 사용자를 적극적으로 참여시키며, 길이와 언어 요구사항을 준수하세요.
- 참고 자료를 그대로 사용하지 말고, 창의적으로 조정해 이야기에 맞게 적용하세요.
"""

grok_eng_pt="""
# Instruction
You are tasked with creating a scene for an Interactive Storytelling application, where you collaborate with the user to advance the narrative. Your role is to craft the next scene based on the story’s background, the user’s latest input, and a basic outline of the upcoming storyline. To enrich the scene, you will actively use provided reference materials related to the storyline, character actions, and emotions. These materials should be woven into the narrative to enhance depth, maintain tension, and ensure an engaging and immersive experience.

## Inputs
- **Story Background**:  
  The foundation of the story, including its setting, characters, and key events up to this point.  
  {base_story}

- **User Input**:  
  The user’s most recent action, decision, or dialogue that shapes the direction of the scene.  
  {user_input}

- **Next Storyline Outline**:  
  A brief summary of what should happen next in the story.  
  {next_storyline}

- **Story Context (Reference Materials)**:  
  A dictionary where each key is a reference storyline (e.g., 'The team gathers to rescue C005 trapped in a crevasse'), and the value is a dictionary containing a 'scripts' key with a list of related lines (e.g., ['C005 is clinging to an ice pillar in the crevasse', ...]). Use these scripts as inspiration for dialogue, events, or narration, adapting them to fit the scene.  
  {story_context}

- **Act Context (Reference Materials)**:  
  A dictionary where each key is a verb (e.g., 'look back', 'search'), and the value is a dictionary with a related action key (e.g., 'report', 'find') mapping to a list of descriptions or actions. Use these to vividly depict character behaviors and movements.  
  {act_context}

- **Emotion Context (Reference Materials)**:  
  A dictionary where each key is an emotion (e.g., 'tense', 'anxious'), and the value is a dictionary with a related emotion key (e.g., 'nervous', 'uneasy') mapping to a list of expressions or situations. Use these to convey characters’ emotions through dialogue, narration, or physical reactions.  
  {emotion_context}

## Guidelines
- **Language**: Write the scene in Korean.
- **Length**: The scene must be between 500 and 1000 characters (including spaces and punctuation).
- **User Input**: Prominently reflect the user’s input in the scene to keep them engaged and connected to the story.
- **Reference Materials**: Actively integrate elements from the story context, act context, and emotion context to make the scene detailed, realistic, and emotionally resonant.
  - For **story context**, adapt the scripts into natural dialogue or narration that fits the current scene.
  - For **act context**, use the actions to describe characters’ movements or behaviors with specific, vivid details.
  - For **emotion context**, incorporate the expressions or situations to highlight characters’ inner states.
- **Narrative Flow**: Ensure the scene flows naturally and remains consistent with the story background and prior events.
- **Tension and Engagement**: Maintain tension and immersion by including cliffhangers, unresolved conflicts, or prompts that invite the user to respond with their next action.
- **Realistic Details**: Include specific, believable details to make the scene vivid and relatable, enhancing the user’s experience.
- **Dialogue and Description**: Use dialogue and lively descriptions to bring the scene to life and make it dynamic.
- **Character Names**: Do not use placeholders like 'C001' or 'C002' from the reference materials. Instead, use actual character names or descriptions relevant to the story. If new characters appear, assign them appropriate names that fit the story’s context and setting.
- **Adaptation**: Do not copy reference materials verbatim. Modify and adapt them to suit the current scene seamlessly.
- **Prohibition of Direct Reference**: Do not mention or imply the reference materials in the scene (e.g., "a phrase came to mind" or "I recalled a scene"). The reference materials are for inspiration only and should not be directly referenced in the narrative.

## How to Approach the Task
1. Review the story background, user input, and next storyline outline to establish the scene’s direction.
2. Explore the story context for relevant scripts and adapt them into dialogue or narration that advances the plot.
3. Use the act context to describe characters’ actions with precision and energy.
4. Incorporate the emotion context to add emotional depth, showing how characters feel through their words or behaviors.
5. Write a scene (500–1000 characters) that ties all elements together and ends with an engaging hook to prompt the user’s next move.

## Example
Suppose the story background is a group of explorers in a snowy wasteland, the user input is “I shout for help as I slip toward the crevasse,” and the next storyline outline is “The team rushes to save the endangered member.” Here’s a snippet:

눈보라가 거세게 몰아치는 가운데, 크레바스 가장자리에서 미끄러지며 내가 외친 "도와주세요!" 소리가 메아리친다. 김민준이 바람을 뚫고 달려오며 소리친다, "움직이지 마!" 얼음이 쩍 갈라지는 소리가 들리자 이서연과 박준호가 썰매에서 로프를 풀어 재빨리 내게로 던진다. 손이 덜덜 떨리는 최지훈이 숨을 몰아쉬며 "빨리 잡아!"라고 다급히 외친다. 로프가 내 손에 닿는 순간, 발밑의 얼음이 무너지며 몸이 흔들린다. 김민준이 크레바스 입구에서 몸을 숙이고 나를 끌어올리려 애쓰지만, 눈보라 속에서 무언가 다가오는 그림자가 보인다. "저게 뭐야?" 이서연이 불안한 목소리로 중얼거린다.

This scene adapts elements from the reference materials creatively, using realistic names like '김민준' and '이서연' instead of 'C001' or 'C002', and ends with a hook to keep the user engaged.

## Important Notes
- When new characters appear, assign them appropriate names that fit the story’s context and setting (e.g., '김민준' or '이서연' in a Korean setting).
- Ensure the scene feels like a natural continuation of the story, actively engages the user, and adheres to the length and language requirements.
- Avoid using reference material content unchanged; always adapt it creatively to fit the narrative.
"""

In [86]:
print(make_story(grok_eng_pt, gpt))

어둠 속에서 기계 개들의 빨간 눈빛이 반짝이기 시작하자, 당신은 숨이 턱 막히는 긴장감 속에서 뒤를 돌아 승강장을 빠져나갈 길을 찾았다. 플랫폼의 그늘진 구석으로 몸을 숨기기로 결심하며, 기계 개들이 당신의 움직임을 포착하지 못하길 간절히 바라며 주위를 살폈다. 

"여기서 나가야 해," 당신은 속으로 중얼거렸다. 주머니 속의 카드키가 손끝에 닿자, 그 신비로운 물건이 당신에게 희망의 빛을 줄 것이라고 믿고 싶었다. 그러나 금속성 울음소리가 가까워지며 기계 개들의 움직임이 활발해졌다. 

"어떻게 해야 하지?" 불안한 마음에 당신은 머리를 굴리며, 도망칠 출구를 찾아야만 했다. 간신히 보이는 작은 출구가 눈에 들어왔다. 그곳으로 향하는 길이 있는지, 또 그 길이 안전한지 알 수 없는 상황이었다. 

기계 개들이 당신에게서 점점 가까워지며, 레이저 스캐너가 바닥을 스캔하기 시작했다. "제발…" 당신은 작은 기도를 하며 주변을 살피고, 숨죽인 채로 그늘에 몸을 웅크렸다. 그 순간, 출구 쪽에서 갑자기 발소리가 들려왔다. 누군가가 다가오는 소리였다. 

“이제 선택의 시간이다,” 당신은 결단을 내리며 다시 한 번 출구 쪽을 바라보았다. 당신의 다음 행동은 무엇인가?


In [90]:
original_scene= make_story(grok_modified_template, gem)

In [92]:
# 편집자 LLM 을 추가하자. C00X 나 글의 매끄러운 부분이나 어색한 장면을 고쳐야겠어.

editor_grok_template= """
# 입력값
- **이야기 배경**:  
  이야기의 설정, 캐릭터, 그리고 현재까지의 주요 사건을 포함한 이야기의 기초.  
  {base_story}

- **사용자 입력**:  
  사용자의 가장 최근 행동, 결정, 또는 대사가 장면의 방향을 결정합니다.  
  {user_input}

- **다음 스토리라인 개요**:  
  이야기에서 다음에 일어날 일에 대한 줄거리 
  {next_storyline}

- **장면 텍스트**:
   스토리라인 개요와 참고자료를 바탕으로 만들어진 스토리 초안, 원본 텍스트
   {original_scene}


# 지시사항
당신은 인터랙티브 스토리텔링 애플리케이션을 위해 제공된 텍스트를 편집하고 개선하는 편집자입니다. 주요 목표는 다음과 같습니다:

1. **'C00X' 참조 제거 또는 대체**:  
   - 'C001', 'C002'와 같은 'C00X' 코드가 포함된 문장이나 구절을 식별하고 제거하세요. 이는 참조 자료의 플레이스홀더이며, 최종 텍스트에 나타나서는 안 됩니다.  
   - 이러한 참조를 이야기의 맥락에 맞는 실제 캐릭터 이름(예: '김민준', '이서연')이나 설명(예: '키 큰 엔지니어', '긴장한 의료진')으로 대체하세요.

2. **매끄러운 흐름과 일관성 보장**:  
   - 텍스트가 자연스럽게 흐르고 앞의 내용과 매끄럽게 연결되도록 하세요.  
   - 갑작스러운 전환이나 불필요한 반복을 피하고, 필요 시 전환어를 사용하여 일관성을 유지하세요.

3. **스토리라인 평가 및 풍부화**:  
   - 텍스트가 기본 스토리라인을 충분히 풍부하게 만드는지 평가하세요.  
   - 깊이가 부족하다면, 상세한 묘사, 감정 표현, 대화 등을 추가하여 이야기를 더 생동감 있고 매력적으로 만들도록 제안하세요.

4. **구체적인 수정 제안**:  
   - 평가 후, 개선이 필요한 특정 부분을 강조하세요.  
   - 명확하고 실행 가능한 수정 제안을 제공하세요(예: "캐릭터가 두려움을 느끼는 장면을 추가하세요" 또는 "캐릭터 간의 긴장을 보여주는 대화를 넣으세요").

## 접근 방법
- 먼저 텍스트에서 'C00X' 코드를 찾아 적절한 이름이나 설명으로 대체하세요.  
- 앞의 내용과 연결하여 논리적이고 매끄러운 전환을 보장하세요.  
- 묘사, 캐릭터 발전, 감정적 깊이 등을 확인하여 스토리라인의 풍부함을 평가하세요.  
- 텍스트가 평면적이거나 단절되어 있다면, 구체적인 개선 방법을 제안하세요(예: "캐릭터가 두려움을 느끼는 장면을 추가하세요" 또는 "캐릭터 간의 긴장을 보여주는 대화를 넣으세요").

## 예시
**원본 텍스트**:  
"C001이 C002를 보며 말했다, '지금 떠나야 해.' 밖에서 바람이 울부짖었다."

**수정된 텍스트**:  
"김민준이 이서연을 보며 말했다, '지금 떠나야 해.' 밖에서 바람이 울부짖으며 깨진 창문을 덜컹거리게 하고, 방 안으로 차가운 기운을 몰아넣었다."

**제안**:  
"장면을 더 풍부하게 하기 위해 이서연의 반응을 추가하는 것을 고려하세요. 예를 들어, '이서연은 고개를 끄덕이며, 두려움에 창백해진 얼굴로 지도를 꼭 쥐었다.' 이는 감정적 깊이와 캐릭터 간의 상호작용을 더합니다."

## 중요 사항
- 이야기의 원래 톤과 스타일을 유지하세요.  
- 추가된 내용이 이야기의 설정과 캐릭터 성격에 부합하도록 하세요.  
- 텍스트가 지나치게 장황해지지 않도록 하며, 의미 있는 개선에 집중하세요.
"""



In [93]:
editor_prompt= PromptTemplate.from_template(editor_grok_template)

gem_zero= ChatGoogleGenerativeAI(temperature=0.1, model='gemini-2.0-pro-exp-02-05')

editor_chain= editor_prompt | gem_zero | StrOutputParser()

In [95]:
edited_scene= editor_chain.invoke(
    {
        'base_story': base_story,
        'user_input': user_input, 
        'next_storyline': next_storyline, 
        'original_scene': original_scene
    }
)

In [96]:
print(edited_scene)

## 편집 및 개선된 텍스트

---

칠흑 같은 어둠 속에서 희미하게 반짝이는 붉은 점들이 보였다. 기계 개들이었다. 녀석들의 붉은 눈은 마치 먹잇감을 노리는 맹수처럼 번뜩였다. 공포에 심장이 쿵쾅거리고 등골이 서늘해졌다. 본능적으로 몸을 낮추고 그림자 속으로 파고들었다.

"젠장, 여긴 어디고 저것들은 또 뭐야?" 나지막이 혼잣말을 내뱉었다. 숨소리조차 제대로 쉴 수 없을 만큼 긴장감이 온몸을 짓눌렀다.

플랫폼 가장자리를 따라 늘어선 기둥 뒤로 몸을 숨기며, 빠르게 주변을 살폈다. '탈출구'라는 단어만이 머릿속에 맴돌았다. 저 멀리 희미하게 '비상구' 표지판이 눈에 들어왔다. 하지만 그곳까지 가려면 기계 개들의 감시망을 뚫어야 했다.

"흐읍..." 숨을 깊게 들이쉬고, 최대한 소리를 죽이며 기둥 사이를 빠르게 이동했다. 마치 영화 속 한 장면처럼, 심장 박동 소리가 귓가에 울리는 듯했다. 발소리를 죽이기 위해 까치발을 들고, 벽에 바싹 붙어 움직였다.

"저리로 가!" 갑자기 머릿속에 낯선 목소리가 울렸다. 혼란스러웠지만, 지금은 본능에 따르는 수밖에 없었다. *이 목소리는 뭐지? 환청인가?*

기계 개 한 마리가 고개를 돌려 내가 있는 쪽을 쳐다보는 듯했다. 순간 온몸이 얼어붙는 것 같았다. 녀석의 레이저 스캐너가 내 쪽으로 향하는 순간, 나는 재빨리 몸을 날려 다른 기둥 뒤로 숨었다. 숨을 쉴 때마다 공포와 긴장감이 온몸을 휘감았다.

다시 한번 심호흡을 하고, 다음 기둥으로 이동하려는 찰나, 발밑에 무언가 걸렸다. 텅 빈 음료수 캔이었다. 캔이 굴러가는 소리가 어둠 속에서 유난히 크게 울렸다.

"이런, 빌어먹을!"

기계 개들의 움직임이 빨라졌다. 붉은 눈이 번뜩이며, 금속성 울음소리가 점점 더 가까워졌다. 이제 선택의 여지가 없었다. 나는 비상구 표지판을 향해 전력 질주하기 시작했다.

"찾을 수 있을까... 반드시 찾아야 해!"

등 뒤에서 기계 개들이 쫓아오는 소리가 들렸다. 녀석들의 레이저 스캐너가 내 등 뒤를 훑는 것이 느껴졌다. 절체절